In [9]:
import cv2
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
import re
from PIL import Image
from pathlib import Path

RAW_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/raw"
STACKREG_DIR = "/Users/farhanoktavian/imperial/thesis/sandbox/dataset/stackreg4"
REG_TYPE = "xd"

In [25]:
sample_id = "HS001"

raw_sample_dir = Path(RAW_DIR) / sample_id
regstack_sample_dir = Path(STACKREG_DIR) / sample_id

raw_runs = sorted([x for x in raw_sample_dir.iterdir() if x.is_dir()])
regstack_runs = sorted([x for x in regstack_sample_dir.iterdir() if x.is_dir()])

In [26]:
run_pairs = []
for run in regstack_runs:
    run_id = run.name
    if run_id in [r.name for r in raw_runs]:
        run_pairs.append((run_id, raw_sample_dir / run_id, run))

In [27]:
def read_image(img_dir):
    img = cv2.imread(str(img_dir), cv2.IMREAD_GRAYSCALE)
    return img

if not Path(f"result/{sample_id}").exists():
    Path(f"result/{sample_id}").mkdir(parents=True)

for run_id, a, b in run_pairs:
    raw_images = [read_image(x) for x in sorted(a.iterdir()) if x.suffix.lower() == ".png"]

    # Need to sort by number
    regstack_paths = sorted(b.iterdir(), key=lambda f: int(re.sub('\D', '', f.name)))
    regstack_images = [read_image(x) for x in regstack_paths if x.suffix.lower() == ".png"]

    if len(raw_images) > 29:
        raw_images = raw_images[::2]
        regstack_images = regstack_images[::2]
    
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    ims = []

    axes[0].imshow(raw_images[0], cmap="gray", vmin=0, vmax=255)
    axes[1].imshow(regstack_images[0], cmap="gray", vmin=0, vmax=255)
    
    def animate(i):
        axes[0].imshow(raw_images[i], cmap="gray", vmin=0, vmax=255, animated=True)
        axes[1].imshow(regstack_images[i], cmap="gray", vmin=0, vmax=255, animated=True)
    
    ani = animation.FuncAnimation(fig, animate, interval=75, blit=False, frames=len(raw_images))
    ani.save(f"result/{sample_id}/{run_id}-{REG_TYPE}.mp4")

plt.close("all")
print("done")

done
